# Классификация комментариев с моделью BERT и без неё (GPU)

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорт компонентов, библиотек и модулей

In [1]:
import pandas as pd
import numpy as np
import time
import lightgbm as lgbm


import tensorflow as tf
from tqdm.notebook import tqdm
import ipyparallel as ipp

import torch
import transformers
import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

import spacy
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.utils import resample

In [2]:
torch.backends.cudnn.enabled

True

In [3]:
torch.cuda.is_available()

True

In [4]:
import multiprocessing
n_jobs = multiprocessing.cpu_count()-1

In [5]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [6]:
torch.cuda.empty_cache()

### Загрузка датасета, изучение данных

In [7]:
try:
    data = pd.read_csv('C:\\Users\\a1\\Desktop\\toxic_comments.csv')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')
data

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
...,...,...,...
159287,159446,""":::::And for the second time of asking, when ...",0
159288,159447,You should be ashamed of yourself \n\nThat is ...,0
159289,159448,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,159449,And it looks like it was actually you who put ...,0


In [8]:
data.shape

(159292, 3)

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [10]:
data.describe()

,Unnamed: 0,toxic
count,159292.000000,159292.000000
mean,79725.697242,0.101612
std,46028.837471,0.302139
min,0.000000,0.000000
25%,39872.750000,0.000000
50%,79721.500000,0.000000
75%,119573.250000,0.000000
max,159450.000000,1.000000


#### Пропуски

Проверим наличие пропусков:

In [11]:
data.isnull().sum()

Unnamed: 0    0
text          0
toxic         0
dtype: int64

Пропусков нет.

#### Дубликаты

Проверим наличие дубликатов:

In [12]:
data.duplicated().sum()

0

Дубликатов нет.

#### Соотношение классов

Определим соотношение классов:

In [13]:
data.query('toxic == 1')
len(data.query('toxic == 1'))/len(data.query('toxic == 0'))

0.11310497114027364

11% комментариев, которые обозначены как "токсичные". Балансировка классов не будет использоваться, во избежание утечки из обучающей выборки в тестовую, так как будем выполнять кросс-валидацию..

#### Удаление не актуальных данных

Избавимся от столбца "Unnamed: 0" с не акуальными данными:

In [14]:
data = data.drop(['Unnamed: 0'], axis=1)

Проверим:

In [15]:
data.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


### Подготовка данных

Для подготовки теста для обучения моделей-регрессоров будем использовать как модель BERT, так и варианты без её применения.

Так как модель BERT использует уже готовые списки для работы с текстом, произведём подготовку для работы с текстом для варианта с использованием векторайзера TF-IDF..

#### Данные для классификации без использования модели BERT

В качестве токенизатора с готовыми списками будем использовать библиотеку spyCy, как более точную альтернативу библиотеки NLTK, изучению которой уделялось внимание в периоде обучения на курсе DS.

Загрузка основной модели английского языка из spaCy:

In [16]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

Зададим стоп-слова библиотеки spaCy:

In [17]:
stop_words = spacy.lang.en.stop_words.STOP_WORDS
print('Number of stop words: %d' % len(stop_words))

Number of stop words: 326


Произведём разбор текстов моделью с последующими токенизацией и лемматизацией текстов:

In [18]:
%%time
tqdm.pandas()
data['final_text'] = data['text'].progress_apply(lambda x: ' '.join([token.lemma_ for token in nlp(x)]))
data.head()

  0%|          | 0/159292 [00:00<?, ?it/s]

Wall time: 18min 19s


,text,toxic,final_text
0,Explanation\nWhy the edits made under my usern...,0,explanation \n why the edit make under my user...
1,D'aww! He matches this background colour I'm s...,0,d'aww ! he match this background colour I be s...
2,"Hey man, I'm really not trying to edit war. It...",0,"hey man , I be really not try to edit war . it..."
3,"""\nMore\nI can't make any real suggestions on ...",0,""" \n More \n I can not make any real suggestio..."
4,"You, sir, are my hero. Any chance you remember...",0,"you , sir , be my hero . any chance you rememb..."


Произведём очистку текста от символов, не являющихся словами:

In [19]:
%%time
tqdm.pandas()
data['final_text'] = data['final_text'].apply(
    lambda x: ' '.join(
        re.sub(
            r'[^a-zA-Z\']',
            ' ',
            x
        ).split()
    )
).str.lower()
data.head()

Wall time: 4.75 s


,text,toxic,final_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,d'aww he match this background colour i be see...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i be really not try to edit war it be ...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can not make any real suggestion on imp...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


Создаём отдельный датасет, в котором оставим только индексы, фичи и таргеты:  

In [20]:
data_1 = data.drop(['text'], axis = 1)

Произведём формирование общей выборки с признаками:

In [21]:
features = data['final_text']
features.head()

0    explanation why the edit make under my usernam...
1    d'aww he match this background colour i be see...
2    hey man i be really not try to edit war it be ...
3    more i can not make any real suggestion on imp...
4    you sir be my hero any chance you remember wha...
Name: final_text, dtype: object

Сформируем общую выборку с целевыми признаками:

In [22]:
target = data['toxic']
target.head()

0    0
1    0
2    0
3    0
4    0
Name: toxic, dtype: int64

Произведём разделение датасета на обучающую и тестовую выборку:

In [23]:
train, test = train_test_split(data_1, test_size=0.1)

Изучим дисбаланс классов:

In [24]:
train_maj = train[train.toxic==0]
train_min = train[train.toxic==1]

In [25]:
train.toxic.value_counts()

0    128824
1     14538
Name: toxic, dtype: int64

Выделим признаки для обучающей и тестовой выборок(включая целевой):

In [26]:
features_train = train['final_text']
target_train = train['toxic']
features_test = test['final_text']
target_test = test['toxic']

Произведём формирование матриц с признаками:

Создим счётчик величин TF-IDF:

In [27]:
count_tf_idf = TfidfVectorizer(
    stop_words=stop_words
)

Выполним подсчёт величин TF-IDF:

In [28]:
tf_idf = count_tf_idf.fit(features_train)

Сформируем матрицы обучаюших признаков с выводом размеров:

In [29]:
features_train_vectorized = count_tf_idf.transform(features_train)
features_train_vectorized.shape

(143362, 148049)

Сформируем матрицы тестовых признаков с выводом размеров:

In [30]:
features_test_vectorized = count_tf_idf.transform(features_test)
features_test_vectorized.shape

(15930, 148049)

Данные для выполнения задачи классификации без использования модели BERT подготовленны. 

#### Данные для классификации с использованием модели BERT

Произведём подготовку данных для модели BERT.

Подготовка включит в себя только начальную очистку текста, лемматизация для модели BERT не требуется.

In [31]:
data_bert = data.copy()

In [32]:
def clear_text(text):
    prom = re.sub(r'[^a-zA-Z]',' ', text).split()
    return ' '.join(prom)

In [33]:
data_bert['clear'] = data_bert['text'].apply(clear_text)

In [34]:
data_bert.head()

,text,toxic,final_text,clear
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,d'aww he match this background colour i be see...,D aww He matches this background colour I m se...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i be really not try to edit war it be ...,Hey man I m really not trying to edit war It s...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can not make any real suggestion on imp...,More I can t make any real suggestions on impr...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...,You sir are my hero Any chance you remember wh...


In [35]:
data_bert = data_bert.drop(['text', 'final_text'], axis=1)
data_bert.head()

,toxic,clear
0,0,Explanation Why the edits made under my userna...
1,0,D aww He matches this background colour I m se...
2,0,Hey man I m really not trying to edit war It s...
3,0,More I can t make any real suggestions on impr...
4,0,You sir are my hero Any chance you remember wh...


Ограничим количество выражений в тексте до 5000 для сокращения времени обработки данных:

In [36]:
data_for_bert = data_bert.sample(5000, random_state=12345).reset_index(drop=True)
data_for_bert.head()

,toxic,clear
0,0,Expert Categorizers Why is there no mention of...
1,1,Noise fart talk
2,0,An indefinite block is appropriate even for a ...
3,0,I don t understand why we have a screenshot of...
4,0,Hello Some of the people places or things you ...


Выполним разбивку на токены и установим ограничение на длину последовательности токенов (для около 400 слов):

In [37]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

tokenized = data_for_bert['clear'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)

Произведём вычисления эмбеддингов для модели BERT:

In [41]:
%%time
model = transformers.BertModel.from_pretrained("bert-base-uncased")
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Wall time: 2.78 s


In [42]:
%%time
batch_size = 128
embeddings = []
for i in tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device)
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch.to(device), attention_mask=attention_mask_batch.to(device))

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy())

  0%|          | 0/39 [00:00<?, ?it/s]

Wall time: 5min 32s


Выделим признаки (включая целевой):

In [43]:
features_bert = np.concatenate(embeddings)

In [44]:
target_bert = data_for_bert['toxic'][0:len(features_bert)]

In [45]:
features_bert.shape

(4992, 768)

In [46]:
target_bert.shape

(4992,)

Сформируем обучающую и тестовую выборки для данных, подготовленных с применениме модели BERT:

In [47]:
features_train_bert, features_test_bert, target_train_bert, target_test_bert = train_test_split(features_bert, 
                                                                                                target_bert, 
                                                                                                test_size=0.1
                                                                                                )

Данные для работы с моделью BERT подготовлены. 

### Вывод

- Загружен датасет, получена общая информация;
- Пропусков и дубликатов нет;
- Проверено соотношение классов;
- Произведена подготовка текста с помощью библиотеки spaCy для выполнения задачи классификации комментариев без использования модели BERT, подготовлены обучающая и тестовые выборки общего датасета, выделены признаки (в том числе целевой).
- Произведена подготовка текста для работы модели BERT, выделены признаки (в том числе целевой).
- Данные для обучения моделей машинного обучения подготовлены.

## Обучение

В качестве моделей машинного обучения мы будем использовать следующие модели-классификаторы:
- LinearRegression;
- RandomForestClassifier;
- CatBoostClassifier;
- LGBMClassifier.

### Классификация без использования модели BERT

Создадим датафрейм, в который будем отправлять модели и значения метрики f1:

In [48]:
results = pd.DataFrame(columns=['Модель', 'Время кросс-валидации и обучения модели, с', 'метрика f1 модели'])

#### LogisticRegression

Обучим логистическую регрессию. Соберём пайплайн, осуществляющий последовательно очистку и векторизацию текста, а затем обучение выбранной модели с заданными гиперпараметрами:

In [49]:
classifier_lr = LogisticRegression()

In [50]:
pipe_lr = Pipeline(
    [
        ('tf_idf', TfidfVectorizer(stop_words=stop_words)),
        ('clf', classifier_lr)
    ]
)

params_lr = {'clf__solver' : ['lbfgs'],
             'clf__C': [15],
             'clf__max_iter':[1000],
             'clf__class_weight': ['balanced']
}

In [51]:
%%time
tqdm.pandas()
start = time.time()
grid_lr = GridSearchCV(pipe_lr,
                       scoring='f1',
                       cv=5,
                       param_grid=params_lr,
                       n_jobs=-1,
                       verbose=2,                       
                       refit=True
)
grid_lr.fit(features_train, target_train)
end = time.time()
f_lr=round(end-start,2)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Wall time: 1min 6s


In [52]:
grid_lr.best_params_

{'clf__C': 15,
 'clf__class_weight': 'balanced',
 'clf__max_iter': 1000,
 'clf__solver': 'lbfgs'}

In [53]:
np.round(grid_lr.best_score_, decimals=2)

0.76

In [54]:
grid_lr.refit_time_

25.157676696777344

In [55]:
results.loc[0] = ['LogisticRegression', f_lr, np.round(grid_lr.best_score_, decimals=2)]

In [56]:
results

,Модель,"Время кросс-валидации и обучения модели, с",метрика f1 модели
0,LogisticRegression,66.81,0.76


#### RandomForestClassifier

In [57]:
classifier_rf = RandomForestClassifier()

In [58]:
pipe_rf = Pipeline(
    [
        ('tf_idf', TfidfVectorizer(stop_words=stop_words)),
        ('clf', classifier_rf)
    ]
)

params_rf = {
             'clf__random_state': [12345],
             'clf__min_samples_leaf': range(2, 10, 2),
             'clf__class_weight': ['balanced'],
             'clf__n_estimators': range(900, 1000, 100),
             'clf__max_depth': range(20, 30, 10)
             }

In [59]:
%%time
start = time.time()
grid_rf = GridSearchCV(pipe_rf,
                       scoring='f1',
                       cv=5,
                       param_grid=params_rf,
                       n_jobs=n_jobs,
                       verbose=3,
                       refit=True
)
grid_rf.fit(features_train, target_train)
end = time.time()
f_rf=round(end-start,2)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Wall time: 5min 43s


In [60]:
grid_rf.best_params_

{'clf__class_weight': 'balanced',
 'clf__max_depth': 20,
 'clf__min_samples_leaf': 2,
 'clf__n_estimators': 900,
 'clf__random_state': 12345}

In [61]:
np.round(grid_rf.best_score_, decimals=2)

0.42

In [62]:
grid_rf.refit_time_

72.50592827796936

In [63]:
results.loc[1] = ['RandomForestClassifier', f_rf, np.round(grid_rf.best_score_, decimals=2)]

In [64]:
results

,Модель,"Время кросс-валидации и обучения модели, с",метрика f1 модели
0,LogisticRegression,66.81,0.76
1,RandomForestClassifier,343.42,0.42


#### LGBMClassifier

In [65]:
torch.cuda.empty_cache()

In [66]:
classifier_lgb = LGBMClassifier()

In [67]:
pipe_lgb = Pipeline(
    [
        ('tf_idf', TfidfVectorizer(stop_words=stop_words)),
        ('clf', classifier_lgb)
    ]
)

params_lgb = {
             'clf__random_state':[12345],
             'clf__boosting_type': ['gbdt'],
             'clf__class_weight': ['balanced'],
             'clf__objective': ['binary'],
             'clf__device_type': ['GPU']
             }

In [68]:
%%time
start = time.time()
grid_lgb = GridSearchCV(pipe_lgb,
                       scoring='f1',
                       cv=5,
                       param_grid=params_lgb,
                       n_jobs=1,
                       verbose=2,
                       refit=True 
)
grid_lgb.fit(features_train, target_train)
end = time.time()
f_lgb=round(end-start,2)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END clf__boosting_type=gbdt, clf__class_weight=balanced, clf__device_type=GPU, clf__objective=binary, clf__random_state=12345; total time=  32.6s
[CV] END clf__boosting_type=gbdt, clf__class_weight=balanced, clf__device_type=GPU, clf__objective=binary, clf__random_state=12345; total time=  30.6s
[CV] END clf__boosting_type=gbdt, clf__class_weight=balanced, clf__device_type=GPU, clf__objective=binary, clf__random_state=12345; total time=  30.0s
[CV] END clf__boosting_type=gbdt, clf__class_weight=balanced, clf__device_type=GPU, clf__objective=binary, clf__random_state=12345; total time=  30.7s
[CV] END clf__boosting_type=gbdt, clf__class_weight=balanced, clf__device_type=GPU, clf__objective=binary, clf__random_state=12345; total time=  29.7s
Wall time: 3min 7s


In [69]:
grid_lgb.best_params_

{'clf__boosting_type': 'gbdt',
 'clf__class_weight': 'balanced',
 'clf__device_type': 'GPU',
 'clf__objective': 'binary',
 'clf__random_state': 12345}

In [70]:
np.round(grid_lgb.best_score_, decimals=2)

0.74

In [71]:
grid_lgb.refit_time_

33.1758759021759

In [72]:
results.loc[2] = ['LGBMClassifier', f_lgb, np.round(grid_lgb.best_score_, decimals=2)]

In [73]:
results

,Модель,"Время кросс-валидации и обучения модели, с",метрика f1 модели
0,LogisticRegression,66.81,0.76
1,RandomForestClassifier,343.42,0.42
2,LGBMClassifier,187.42,0.74


#### CatBoostClassifier

In [74]:
torch.cuda.empty_cache()

In [75]:
classifier_cb = CatBoostClassifier()

In [76]:
pipe_cb = Pipeline(
    [
        ('tf_idf', TfidfVectorizer(stop_words=stop_words)),
        ('clf', classifier_cb)
    ]
)
params_cb = {
             'clf__random_state':[12345],
             'clf__iterations': [200],
             'clf__eval_metric': ['F1'],
             'clf__learning_rate': [0.1],  
             'clf__task_type': ['GPU']
             } 

In [77]:
%%time
start = time.time()
grid_cb = GridSearchCV(pipe_cb,
                       scoring='f1',
                       cv=5,
                       param_grid=params_cb,
                       n_jobs=1,
                       verbose=2,
                       refit=True
)
grid_cb.fit(features_train, target_train)
end = time.time()
f_cb=round(end-start,2)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
0:	learn: 0.5318994	total: 1.34s	remaining: 4m 26s
1:	learn: 0.4659703	total: 2.25s	remaining: 3m 43s
2:	learn: 0.5288081	total: 3.17s	remaining: 3m 28s
3:	learn: 0.5037946	total: 4.07s	remaining: 3m 19s
4:	learn: 0.4903588	total: 4.96s	remaining: 3m 13s
5:	learn: 0.5320307	total: 5.87s	remaining: 3m 9s
6:	learn: 0.5504074	total: 6.76s	remaining: 3m 6s
7:	learn: 0.5163309	total: 7.66s	remaining: 3m 3s
8:	learn: 0.5192116	total: 8.55s	remaining: 3m 1s
9:	learn: 0.5165289	total: 9.45s	remaining: 2m 59s
10:	learn: 0.5011430	total: 10.3s	remaining: 2m 57s
11:	learn: 0.5019359	total: 11.2s	remaining: 2m 55s
12:	learn: 0.5032659	total: 12.1s	remaining: 2m 53s
13:	learn: 0.5057588	total: 12.9s	remaining: 2m 52s
14:	learn: 0.5272920	total: 13.9s	remaining: 2m 51s
15:	learn: 0.5450086	total: 14.8s	remaining: 2m 49s
16:	learn: 0.5311569	total: 15.6s	remaining: 2m 48s
17:	learn: 0.5298030	total: 16.6s	remaining: 2m 47s
18:	learn: 0.530774

In [78]:
grid_cb.best_params_

{'clf__eval_metric': 'F1',
 'clf__iterations': 200,
 'clf__learning_rate': 0.1,
 'clf__random_state': 12345,
 'clf__task_type': 'GPU'}

In [79]:
np.round(grid_cb.best_score_, decimals=2)

0.7

In [80]:
grid_cb.refit_time_

647.278243303299

In [81]:
results.loc[3] = ['CatBoostClassifier', f_cb, np.round(grid_cb.best_score_, decimals=2)]

In [82]:
results

,Модель,"Время кросс-валидации и обучения модели, с",метрика f1 модели
0,LogisticRegression,66.81,0.76
1,RandomForestClassifier,343.42,0.42
2,LGBMClassifier,187.42,0.74
3,CatBoostClassifier,2570.36,0.70


#### Общий результат

Сведём полученные данные в одну таблицу.

In [83]:
results

,Модель,"Время кросс-валидации и обучения модели, с",метрика f1 модели
0,LogisticRegression,66.81,0.76
1,RandomForestClassifier,343.42,0.42
2,LGBMClassifier,187.42,0.74
3,CatBoostClassifier,2570.36,0.70


Из сводной таблицы видно, что моделью с наилушим значением метрики f1 на обучающей выборке, сформированной с использованием библиотеки spaCy и векторизованной с помощью TfidfVectorizer является модель LogisticRegression.

### Классификация с использованием модели BERT

Создадим датафрейм, в который будем отправлять модели и значения метрики f1:

In [84]:
results2 = pd.DataFrame(columns=['Модель', 'Время кросс-валидации и обучения модели, с', 'метрика f1 модели'])

#### LogisticRegression

In [85]:
classifier_lr_bert = LogisticRegression()

In [86]:
pipe_lr_bert = Pipeline(
    [('clf', classifier_lr_bert)]
)

In [87]:
params_lr_bert = {'clf__solver' : ['lbfgs'],
                  'clf__random_state': [12345],
                  'clf__C': [15],
                  'clf__max_iter':[100, 1000, 100],
                  'clf__class_weight': ['balanced']
}

In [88]:
%%time
start = time.time()
grid_lr_bert = GridSearchCV(pipe_lr_bert,
                       scoring='f1',
                       cv=5,
                       param_grid=params_lr_bert,
                       n_jobs=n_jobs,
                       verbose=2,
                       refit=True
)
grid_lr_bert.fit(features_train_bert, target_train_bert)
end = time.time()
f_lr_bert=round(end-start,2)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Wall time: 19.9 s


In [89]:
grid_lr_bert.best_params_

{'clf__C': 15,
 'clf__class_weight': 'balanced',
 'clf__max_iter': 100,
 'clf__random_state': 12345,
 'clf__solver': 'lbfgs'}

In [90]:
np.round(grid_lr_bert.best_score_, decimals=2)

0.67

In [91]:
grid_lr_bert.refit_time_

0.662980318069458

In [92]:
results2.loc[0] = ['LogisticRegression', f_lr_bert, np.round(grid_lr_bert.best_score_, decimals=2)]

In [93]:
results2

,Модель,"Время кросс-валидации и обучения модели, с",метрика f1 модели
0,LogisticRegression,19.9,0.67


#### RandomForestClassifier

In [94]:
classifier_rf_bert = RandomForestClassifier()

In [95]:
pipe_rf_bert = Pipeline(
    [('clf', classifier_rf_bert)]
)

In [96]:
params_rf_bert = {
             'clf__random_state': [12345],
             'clf__min_samples_leaf': range(2, 10, 2),
             'clf__class_weight': ['balanced'],
             'clf__n_estimators': range(900, 1000, 100),
             'clf__max_depth': range(20, 30, 10)
             }

In [97]:
%%time
start = time.time()
grid_rf_bert = GridSearchCV(pipe_rf_bert,
                       scoring='f1',
                       cv=5,
                       param_grid=params_rf_bert,
                       n_jobs=n_jobs,
                       verbose=2,
                       refit=True
)
grid_rf_bert.fit(features_train_bert, target_train_bert)
end = time.time()
f_rf_bert=round(end-start,2)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Wall time: 2min 15s


In [98]:
grid_rf_bert.best_params_

{'clf__class_weight': 'balanced',
 'clf__max_depth': 20,
 'clf__min_samples_leaf': 8,
 'clf__n_estimators': 900,
 'clf__random_state': 12345}

In [99]:
np.round(grid_rf_bert.best_score_, decimals=2)

0.57

In [100]:
grid_rf_bert.refit_time_

44.72452187538147

In [101]:
results2.loc[1] = ['RandomForestClassifier', f_rf_bert, np.round(grid_rf_bert.best_score_, decimals=2)]

In [102]:
results2

,Модель,"Время кросс-валидации и обучения модели, с",метрика f1 модели
0,LogisticRegression,19.90,0.67
1,RandomForestClassifier,135.45,0.57


#### LGBMClassifier

In [103]:
torch.cuda.empty_cache()

In [104]:
classifier_lgb_bert = LGBMClassifier()

In [105]:
pipe_lgb_bert = Pipeline(
    [('clf', classifier_lgb_bert)]
)

In [106]:
params_lgb_bert = {
             'clf__random_state':[12345],
             'clf__boosting_type': ['gbdt'],
             'clf__class_weight': ['balanced'],
             'clf__objective': ['binary'],
             'clf__device_type': ['GPU']
}

In [107]:
%%time
start = time.time()
grid_lgb_bert = GridSearchCV(pipe_lgb_bert,
                       scoring='f1',
                       cv=5,
                       param_grid=params_lgb_bert,
                       n_jobs=n_jobs,
                       verbose=2,
                       refit=True 
)
grid_lgb_bert.fit(features_train_bert, target_train_bert)
end = time.time()
f_lgb_bert=round(end-start,2)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Wall time: 35.4 s


In [108]:
grid_lgb_bert.best_params_

{'clf__boosting_type': 'gbdt',
 'clf__class_weight': 'balanced',
 'clf__device_type': 'GPU',
 'clf__objective': 'binary',
 'clf__random_state': 12345}

In [109]:
np.round(grid_lgb_bert.best_score_, decimals=2)

0.63

In [110]:
grid_lgb_bert.refit_time_

5.168255567550659

In [111]:
results2.loc[2] = ['LGBMClassifier', f_lgb_bert, np.round(grid_lgb_bert.best_score_, decimals=2)]

In [112]:
results2

,Модель,"Время кросс-валидации и обучения модели, с",метрика f1 модели
0,LogisticRegression,19.90,0.67
1,RandomForestClassifier,135.45,0.57
2,LGBMClassifier,35.38,0.63


#### CatBoostClassifier

In [113]:
torch.cuda.empty_cache()

In [114]:
classifier_cb_bert = CatBoostClassifier()

In [115]:
pipe_cb_bert = Pipeline(
    [('clf', classifier_cb_bert)]
)

In [116]:
params_cb_bert = {
             'clf__random_state':[12345],
             'clf__iterations': [200],
             'clf__eval_metric': ['F1'],
             'clf__learning_rate': [0.1],  
             'clf__task_type': ['GPU'],
              'clf__verbose': [500]
             }

In [117]:
%%time
start = time.time()
grid_cb_bert = GridSearchCV(pipe_cb_bert,
                       scoring='f1',
                       cv=5,
                       param_grid=params_cb_bert,
                       n_jobs=1,
                       verbose=2,
                       refit=True
)
grid_cb_bert.fit(features_train_bert, target_train_bert)
end = time.time()
f_cb_bert=round(end-start,2)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
0:	learn: 0.5090253	total: 116ms	remaining: 23s
199:	learn: 1.0000000	total: 27.3s	remaining: 0us
[CV] END clf__eval_metric=F1, clf__iterations=200, clf__learning_rate=0.1, clf__random_state=12345, clf__task_type=GPU, clf__verbose=500; total time=  28.9s
0:	learn: 0.4476386	total: 126ms	remaining: 25.1s
199:	learn: 1.0000000	total: 27.7s	remaining: 0us
[CV] END clf__eval_metric=F1, clf__iterations=200, clf__learning_rate=0.1, clf__random_state=12345, clf__task_type=GPU, clf__verbose=500; total time=  29.3s
0:	learn: 0.5470693	total: 110ms	remaining: 21.9s
199:	learn: 1.0000000	total: 29.1s	remaining: 0us
[CV] END clf__eval_metric=F1, clf__iterations=200, clf__learning_rate=0.1, clf__random_state=12345, clf__task_type=GPU, clf__verbose=500; total time=  30.7s
0:	learn: 0.5121495	total: 130ms	remaining: 25.9s
199:	learn: 1.0000000	total: 29.1s	remaining: 0us
[CV] END clf__eval_metric=F1, clf__iterations=200, clf__learning_rate=0.

In [118]:
grid_cb_bert.best_params_

{'clf__eval_metric': 'F1',
 'clf__iterations': 200,
 'clf__learning_rate': 0.1,
 'clf__random_state': 12345,
 'clf__task_type': 'GPU',
 'clf__verbose': 500}

In [119]:
np.round(grid_cb_bert.best_score_, decimals=2)

0.59

In [120]:
grid_cb_bert.refit_time_

29.349868774414062

In [121]:
results2.loc[3] = ['CatBoostClassifier', f_cb_bert, np.round(grid_cb_bert.best_score_, decimals=2)]

In [122]:
results2

,Модель,"Время кросс-валидации и обучения модели, с",метрика f1 модели
0,LogisticRegression,19.90,0.67
1,RandomForestClassifier,135.45,0.57
2,LGBMClassifier,35.38,0.63
3,CatBoostClassifier,179.53,0.59


#### Общий результат

In [123]:
results2

,Модель,"Время кросс-валидации и обучения модели, с",метрика f1 модели
0,LogisticRegression,19.90,0.67
1,RandomForestClassifier,135.45,0.57
2,LGBMClassifier,35.38,0.63
3,CatBoostClassifier,179.53,0.59


Относительно не высокие показатели результативности по метрике f1 вызваны тем, что используется модель BERT, обученная на обычных текстах, доступных из публичных источниках, для которой не ставилась задача оценки комментариев на токсичность (не приемлиемый контент). Для таких задач используется "toxic BERT" и ей подобные узко специализированные модели NLP. 

### Обобщённые данные и проверка на тестовой выборке

Обобщим данные по моделям машинного обучения, обученных на данных, подготовленных без использования модели BERT:

In [124]:
space = {'test':['TF-IDF', 'TF-IDF', 'TF-IDF', 'TF-IDF', 'BERT', 'BERT', 'BERT', 'BERT'],
         'model':['LogisticRegression', 'RandomForestClassifier', 'LGBMClassifier', 'CatBoostClassifier']*2,
         'f1':[np.round(grid_lr.best_score_, decimals=2),
               np.round(grid_rf.best_score_, decimals=2), 
               np.round(grid_lgb.best_score_, decimals=2), 
               np.round(grid_cb.best_score_, decimals=2), 
               np.round(grid_lr_bert.best_score_, decimals=2), 
               np.round(grid_rf_bert.best_score_, decimals=2),
               np.round(grid_lgb_bert.best_score_, decimals=2),
               np.round(grid_cb_bert.best_score_, decimals=2)]
         }
res = pd.DataFrame(data=space)
res

,test,model,f1
0,TF-IDF,LogisticRegression,0.76
1,TF-IDF,RandomForestClassifier,0.42
2,TF-IDF,LGBMClassifier,0.74
3,TF-IDF,CatBoostClassifier,0.70
4,BERT,LogisticRegression,0.67
5,BERT,RandomForestClassifier,0.57
6,BERT,LGBMClassifier,0.63
7,BERT,CatBoostClassifier,0.59


Проверка лучшей модели на тестовой выборке:

In [125]:
%%time
start = time.time()
predicted_lr = grid_lr.predict(features_test)
end = time.time()
t_p_lr=round(end-start,2)

Wall time: 971 ms


In [127]:
print(f1_score(target_test, predicted_lr))

0.7578768095373262


### Вывод

По итогам лучшей моделью машинного обучения для классификации комментариев оказалась модель LogisticRegression (f1=0.76), данные для которой были получены с помощью библиотеки spaCy и векторизованны с помощью TfidfVectorizer. На тестовой выборке эта модель машинного обучения показала метрику f1=0.76, удовлетворяющую условиям выполнения данного проекта. 

## Выводы

- Загружен датасет, получена общая информация;
- Пропусков и дубликатов нет;
- Проверено соотношение классов;
- Произведена подготовка текста с помощью библиотеки spaCy для выполнения задачи классификации комментариев без использования модели BERT, подготовлены обучающая и тестовые выборки общего датасета, произведена балансировка классов методом upsampling, выделены признаки (в том числе целевой).
- Произведена подготовка текста для работы модели BERT, выделены признаки (в том числе целевой).
- Данные для обучения моделей машинного обучения подготовлены;
- Выполнено обучение моделей-классификаторов машинного обучения на данных подготовленных как с помощью библиотеки spaCy и векторайзера TF-IDF Vectorizer, так и модели BERT;
- Наилучший результат по метрике f1 на обучающей выборке был получен на моделе-классификаторе LogisticRegression, использовавшей данные, подготовленные с помощью библиотеки spaCy и векторайзера TF-IDF Vectorizer. Результат был подтверждён на тестовой выборке, установленному заданием критерию соотвтетствует.    

### Бонус

<b> Comme un bonus pour les amis )

Определим на практике как работает лучшая модель.

Отдельный файл содержит фразу, которую обученная с помощью модели BERT модель машинного обучения, показавшая лучший результат по метрике F1, должна оценить на степень приемлиемости (в рамках цензуры). 

Фраза, записанная в файл: "I have no idea what is happening with to the Jupyter network stack in Yandex Practicum, but it looks like a classic German porn: there's a hell of a lot going on around here, everyone is naked, only wearing socks and shouting "ja ja die est Schtangencircul!!!" loudly."

In [360]:
data1 = pd.read_excel(r'C:\Users\a1\Desktop\1.xls')

In [361]:
print(data1)

                                                text  toxic
0  I have no idea what is happening with to the J...    NaN


In [362]:
new_data = pd.DataFrame(data1)

In [363]:
new_data

,text,toxic
0,I have no idea what is happening with to the J...,NaN


Почистим текст:

In [364]:
def clear_text(text):
    prom = re.sub(r'[^a-zA-Z]',' ', text).split()
    return ' '.join(prom)

In [367]:
new_data

,text,toxic
0,I have no idea what is happening with to the J...,NaN


In [368]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')

tokenized = new_data['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

attention_mask = np.where(padded != 0, 1, 0)
padded

array([[  101,  1045,  2031,  2053,  2801,  2054,  2003,  6230,  2007,
         2000,  1996, 13035,  2897,  9991,  1999, 13619,  3207,  2595,
        10975, 28804,  2819,  1010,  2021,  2009,  3504,  2066,  1037,
         4438,  2446, 22555,  1024,  2045,  1005,  1055,  1037,  3109,
         1997,  1037,  2843,  2183,  2006,  2105,  2182,  1010,  3071,
         2003,  6248,  1010,  2069,  4147, 14829,  1998, 11273,  1000,
        14855, 14855,  3280,  9765,  8040, 22893, 25997,  6895, 11890,
         5313,   999,   999,   999,  1000,  9928,  1012,   102]])

In [369]:
%%time
model = transformers.BertModel.from_pretrained("bert-base-uncased")
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Wall time: 2.11 s


In [370]:
padded.shape[0]

1

In [371]:
%%time
batch_size = 1
embeddings = []
for i in tqdm(range(padded.shape[0] // batch_size)):
    batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]).to(device)
    attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

    with torch.no_grad():
        batch_embeddings = model(batch.to(device), attention_mask=attention_mask_batch.to(device))

    embeddings.append(batch_embeddings[0][:,0,:].cpu().numpy()) 
embeddings

  0%|          | 0/1 [00:00<?, ?it/s]

Wall time: 228 ms


[array([[ 2.96792686e-01,  2.79288054e-01,  3.51119369e-01,
         -4.09465820e-01, -3.75003040e-01, -8.26261699e-01,
          5.35689831e-01,  7.81119049e-01,  2.26341590e-01,
         -2.01149598e-01,  2.06364710e-02,  5.54716662e-02,
         -5.84685728e-02,  4.49651003e-01,  5.59139192e-01,
          3.64042729e-01, -4.11000341e-01,  4.29950148e-01,
          2.52891153e-01,  1.12101182e-01, -5.95120192e-02,
         -2.35202625e-01,  1.22811645e-01,  4.47235517e-02,
          2.11547375e-01, -1.14213668e-01,  1.32680163e-02,
         -2.03619555e-01, -2.65167683e-01,  1.16182722e-01,
         -2.42537912e-02,  2.15747207e-01,  4.34028188e-04,
         -1.02725469e-01,  3.16843778e-01, -2.74089687e-02,
          3.23940329e-02,  5.68785518e-03,  5.07611752e-01,
         -6.13097697e-02, -1.97910652e-01, -6.45651110e-03,
          4.97508235e-02, -8.38904157e-02, -2.24781364e-01,
         -6.99066520e-01, -3.51534247e+00,  5.61960079e-02,
         -1.95572793e-01, -2.43622035e-0

In [372]:
features_n = np.concatenate(embeddings)

Эти признаки будут использованы в качестве тестовых. 

Произведём оценку токсичности комментария. Для этого мы будем использовать модель LogisticRegression для варианта с BERT,  уже прошедшей обучение по определению токсичности комментариев на обучающей выборке.

In [379]:
%%time
start = time.time()
test_predict_lr_n = grid_lr_bert.predict(features_n)
end = time.time()
t_p_lr_n =round(end-start,2)

Wall time: 0 ns


In [384]:
if int(test_predict_lr_n) == 1:
    print("Фраза является токсичной")
else:
    print("Фраза не является токсичной")

Фраза является токсичной
